# DATA CLEANING

## Loading the dataset

First things first, we will import the required libraries

In [ ]:
# import pandas
import pandas as pd

Now, we will load the quakes dataset into the notebook

In [ ]:
# load the dataset
df= pd.read_csv('quakes.csv')

Let us see the first few rows in the dataset

In [ ]:
df.head()

Now, we will access the shape of the dataset

In [ ]:
# check the shape of the dataset
df.shape

**We have 19,244 rows and 22 columns in the dataset**

Let's examine the data types of the columns in the dataset.

In [ ]:
df.info()

It is evident that the dataset contains null values, and the data types of some columns are not appropriate.

## Data Cleaning

### Step 1: Converting the 'Time' and 'Updated' Column to Datetime

In the initial data cleaning step, we will convert the "time" and "updated" column to a datetime data type.

In [ ]:
# Convert the 'time' and 'updated' column to datetime
df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])

In [ ]:
df.info()

**We have converted 'time' and 'updated' columns from the object data type to datetime64[ns, UTC]. The data now reflects a more appropriate datetime format for further analysis and processing.**


### Step 2: Setting Time Column as Index

In the second data cleaning step, we will set the time column as the new index for the dataset.

In [ ]:
# set time colum as index
df.index = df.time

In [ ]:
df.head()

In [ ]:
# change the index name to 'time_index'
df.index.name = 'time'
df.head()

Great!!!

### Step 3: Handling Null Values

In this stage of data cleaning, we will address and handle all null values present in the dataset.

Let's identify which datasets within the dataset contain null values.

In [ ]:
# checking for nulls
df.isnull().sum()

9 columns contain null values in the dataset

### a. Dealing with nulls in the 'place' column

For null values in the "place" column, we will substitute them with 'Others' to denote alternative locations.

In [ ]:
df['place'] = df['place'].fillna('Others')

In [ ]:
# checking for nulls
df.isnull().sum()

Good!!!

### b. Dealing with nulls in the other columns

Now, we will deal with the nulls in other columns. Since the remaining columns are all quantitative variable, we will fill all the nulls with zero.

In [ ]:
# List of columns to fill null values
columns_to_fill = ['nst', 'gap', 'dmin', 'horizontalError', 'depthError', 'magError', 'magNst', 'rms']

# Iterate through each column and fill null values with 0
for column in columns_to_fill:
    # Fill null values in the specified column with 0
    df[column] = df[column].fillna(0)


In [ ]:
# checking for nulls
df.isnull().sum()

**Excellent! We have effectively addressed all null values.**

### Step 4: Converting columns to appropriate data types

Now, we will attempt to convert the various columns into the appropriate data types.

In [ ]:
df.info()

We will convert the 'magType', 'net', 'id', 'place', 'type', 'status', 'locationSource', 'magSource', and 'state' column from object to string data type

In [ ]:
# List of columns to convert to string data type
columns_to_convert_to_string = ['magType', 'net', 'id', 'place', 'type', 'status', 'locationSource', 'magSource']

# Convert specified columns to string data type
df[columns_to_convert_to_string] = df[columns_to_convert_to_string].astype('string')


In [ ]:
df.info()

Next we will convert the 'nst' and 'magNst'column into integers

In [ ]:
# List of columns to convert to integer data type
columns_to_convert_to_int = ['nst', 'magNst']

# Convert specified columns to integer data type
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].astype(int)


In [ ]:
df.info()

**Great. We have successfully converted all columns to their appropriate data types.**

### Step 5: Obtaining the state and country of the seismic events

We will enhance our dataset by extracting precise state and country information from the 'longitude' and 'latitude column using the Geopy Python library. This library utilizes longitude and latitude coordinates to generate the corresponding state and country names. Consequently, we will create a new column named 'state_country' to capture this geographically derived information.

In [ ]:
df.head(1)

As an example, we will use the library to obtain the exact state and location of the first seismic event in the datatset

In [ ]:
# import the required library
from geopy.geocoders import Nominatim

# Create a Nominatim geolocator
geolocator = Nominatim(user_agent="reverse_geocoding_example")

# coordinates of the first seismic event
latitude = 61.5808
longitude = -149.847

# Perform reverse geocoding
location = geolocator.reverse((latitude, longitude), language='en')

if location is not None:
    address = location.address
    print(f"The location at ({latitude}, {longitude}) is: {address}")
else:
    print(f"No location found for coordinates ({latitude}, {longitude})")


**This shows the exact location of the first seismic event**

We will now replicate the process to extract the state and country information for all seismic events. To mitigate runtime errors, we will parse in the coordinates of 50 events at a time.

**Disclaimer: This body of code might take about 2 hours to run, due to numerous request sent to Geopy**

In [ ]:
# import tqdm to show the progress bar
from tqdm import tqdm  

# Create a Nominatim geolocator
geolocator = Nominatim(user_agent="reverse_geocoding_examples")


# Function to perform reverse geocoding for a batch of rows
def reverse_geocode_batch(batch):
    results = []
    for index, row in batch.iterrows():
        location = geolocator.reverse((row['latitude'], row['longitude']), language='en')
        if location is not None:
            results.append(location.address)
        else:
            results.append("No location found")
    return results

# make a copy of the dataset
df_reset = df

# Reset the index before processing in batches
df_reset = df.reset_index(drop=True)

# Process the DataFrame in batches of 50 rows
batch_size = 50
num_batches = len(df_reset) // batch_size + 1

 
# Enable progress bar for pandas
tqdm.pandas()

# Apply the function to create a new column 'state_country'
df_reset['state_country'] = df_reset.groupby(df_reset.index // batch_size).progress_apply(reverse_geocode_batch).explode().reset_index(drop=True)

# Print the updated DataFrame
df_reset


In [ ]:
# set time column as index
df_reset.index = df_reset.time

In [ ]:
# Convert state_country columns to string data type
df_reset['state_country'] = df_reset['state_country'].astype('string')

In [ ]:
df_reset.head()

In [ ]:
df_reset.info()

### Saving the dataset

Finally, we will save the cleaned dataset as a CSV file called 'quakes-cleaned.csv'

In [ ]:
# save as csv
df_reset.to_csv('quakes-cleaned.csv')